In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

In [2]:
import nltk
# nltk.download()

In [3]:
df = pd.read_csv('D:\LYS\Team Project2(빅데이터를 활용한 소방안전에 대한 동향분석 및 정책이슈 도출)-20221211T134252Z-001\Data\연합뉴스API\df100000.csv', encoding = 'utf-8')
df

,Unnamed: 0,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,...,YNewsML.Metadata.Region.RegionCode,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.RefContent,YNewsML.Metadata.Issue.IssueCode.code,YNewsML.Metadata.Issue.IssueCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Embargo
0,0,rens_test,_doc,AKR20210105121500009,NaN,160431,C,AKR0,AKR20210105121500009,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,rens_test,_doc,AKR20210105124700009,NaN,160508,C,AKR0,AKR20210105124700009,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,rens_test,_doc,AKR20210105123300530,NaN,160514,C,AKR0,AKR20210105123300530,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,rens_test,_doc,AKR20210105126900002,NaN,160538,C,AKR0,AKR20210105126900002,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,rens_test,_doc,AKR20210105126800017,NaN,160650,C,AKR0,AKR20210105126800017,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109995,109995,rens_test,_doc,PYH20210406053100013,NaN,104446,C,PYH0,PYH20210406053100013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109996,109996,rens_test,_doc,PYH20210406053200013,NaN,104450,C,PYH0,PYH20210406053200013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109997,109997,rens_test,_doc,PYH20210406053300013,NaN,104457,C,PYH0,PYH20210406053300013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109998,109998,rens_test,_doc,AKR20210406067000001,NaN,104505,C,AKR0,AKR20210406067000001,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns.tolist()

['Unnamed: 0',
 '_index',
 '_type',
 '_id',
 '_score',
 'YNewsML.Header.SendTime',
 'YNewsML.Header.Action',
 'YNewsML.Header.ServiceType',
 'YNewsML.Header.ContentID',
 'YNewsML.Header.SendDate',
 'YNewsML.Metadata.Category.code',
 'YNewsML.Metadata.Category.name',
 'YNewsML.Metadata.Credit',
 'YNewsML.Metadata.DeliCode',
 'YNewsML.Metadata.Href',
 'YNewsML.Metadata.Desk.code',
 'YNewsML.Metadata.Desk.content',
 'YNewsML.Metadata.Class.ClassCode',
 'YNewsML.Metadata.Urgency',
 'YNewsML.Metadata.Source',
 'YNewsML.Metadata.Writer.uid',
 'YNewsML.Metadata.Writer.content',
 'YNewsML.NewsContent.LangType',
 'YNewsML.NewsContent.Title',
 'YNewsML.NewsContent.Body',
 'send_timestamp',
 'YNewsML.Metadata.Region.RegionCode.code',
 'YNewsML.Metadata.Region.RegionCode.name',
 'YNewsML.NewsContent.SubTitle',
 'YNewsML.Metadata.Attribute.AttributeCode.code',
 'YNewsML.Metadata.Attribute.AttributeCode.name',
 'YNewsML.Metadata.Class.ClassCode.code',
 'YNewsML.Metadata.Class.ClassCode.name',
 'YNew

In [5]:
cond1 = df['YNewsML.Metadata.Desk.content'] == '사진부'

df1 = df.loc[cond1]
df2 = df.loc[~cond1]

In [6]:
df1

,Unnamed: 0,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,...,YNewsML.Metadata.Region.RegionCode,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.RefContent,YNewsML.Metadata.Issue.IssueCode.code,YNewsML.Metadata.Issue.IssueCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Embargo
5,5,rens_test,_doc,PYH20210105162100013,NaN,160736,C,PYH0,PYH20210105162100013,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,rens_test,_doc,PYH20210105152300007,NaN,162559,C,PYH0,PYH20210105152300007,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,17,rens_test,_doc,PYH20210105156900007,NaN,162911,C,PYH0,PYH20210105156900007,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,rens_test,_doc,PYH20210105164300013,NaN,163246,C,PYH0,PYH20210105164300013,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,rens_test,_doc,PYH20210105164500013,NaN,163307,C,PYH0,PYH20210105164500013,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109994,109994,rens_test,_doc,PYH20210406053000013,NaN,104443,C,PYH0,PYH20210406053000013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109995,109995,rens_test,_doc,PYH20210406053100013,NaN,104446,C,PYH0,PYH20210406053100013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109996,109996,rens_test,_doc,PYH20210406053200013,NaN,104450,C,PYH0,PYH20210406053200013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109997,109997,rens_test,_doc,PYH20210406053300013,NaN,104457,C,PYH0,PYH20210406053300013,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df2

,Unnamed: 0,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,...,YNewsML.Metadata.Region.RegionCode,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.RefContent,YNewsML.Metadata.Issue.IssueCode.code,YNewsML.Metadata.Issue.IssueCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Embargo
0,0,rens_test,_doc,AKR20210105121500009,NaN,160431,C,AKR0,AKR20210105121500009,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,rens_test,_doc,AKR20210105124700009,NaN,160508,C,AKR0,AKR20210105124700009,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,rens_test,_doc,AKR20210105123300530,NaN,160514,C,AKR0,AKR20210105123300530,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,rens_test,_doc,AKR20210105126900002,NaN,160538,C,AKR0,AKR20210105126900002,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,rens_test,_doc,AKR20210105126800017,NaN,160650,C,AKR0,AKR20210105126800017,20210105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109986,109986,rens_test,_doc,AKR20210406067700054,NaN,104329,C,AKR0,AKR20210406067700054,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109989,109989,rens_test,_doc,AKR20210406068100002,NaN,104355,C,AKR0,AKR20210406068100002,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109990,109990,rens_test,_doc,AKR20210406063000001,NaN,104356,C,AKR0,AKR20210406063000001,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109992,109992,rens_test,_doc,AKR20210406059700001,NaN,104417,C,AKR0,AKR20210406059700001,20210406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.isnull().sum()

Unnamed: 0                                               0
_index                                                   0
_type                                                    0
_id                                                      0
_score                                              110000
YNewsML.Header.SendTime                                  0
YNewsML.Header.Action                                    0
YNewsML.Header.ServiceType                               0
YNewsML.Header.ContentID                                 0
YNewsML.Header.SendDate                                  0
YNewsML.Metadata.Category.code                       22677
YNewsML.Metadata.Category.name                       22677
YNewsML.Metadata.Credit                                  0
YNewsML.Metadata.DeliCode                            59452
YNewsML.Metadata.Href                                    0
YNewsML.Metadata.Desk.code                               0
YNewsML.Metadata.Desk.content                           

In [44]:
df['YNewsML.NewsContent.SubTitle'].value_counts()

전남 서해 먼바다에 풍랑주의보 해제                                                     6
홍남기 부총리, 부동산 투기근절·재발방지대책 브리핑\r\n(끝)                                     5
이탈리아, 덴마크, 노르웨이 이어 10여개국 중단…"중단은 실책" 비판도 제기                             4
국회, 4차 재난지원금 포함 추가경정예산안 처리\r\n(끝)                                       3
금융권 두번째 4연임 회장…하나은행 등 5개사 차기 CEO 내일 결정\n지성규 하나은행장·이진국 하나금투 대표 교체 가능성    3
                                                                       ..
전남 서해안 등 4곳 풍랑주의보 해제                                                    1
금융당국 "명백히 비합리적이지 않다면 회계 오류 아냐"\r\n코로나19 상황에서의 자산손상 감독지침 거듭 강조           1
외교부 "긴밀한 소통과 상호이해, 양국관계 발전에 도움"                                         1
지역주민들, 권익위에 고충민원…조정기간 동안 훈련않기로                                          1
"투표는 세상을 바꿀 소중한 권리…꼭 행사해 달라"                                            1
Name: YNewsML.NewsContent.SubTitle, Length: 23579, dtype: int64

In [ ]:
df2[
 'YNewsML.Metadata.Region.RegionCode.code',
 'YNewsML.Metadata.Region.RegionCode.name',
 'YNewsML.NewsContent.SubTitle',
 'YNewsML.Metadata.Attribute.AttributeCode.code',
 'YNewsML.Metadata.Attribute.AttributeCode.name',
 'YNewsML.Metadata.Class.ClassCode.code',
 'YNewsML.Metadata.Class.ClassCode.name',
 'YNewsML.Metadata.Category',
 'YNewsML.Metadata.PublishNo',
 'YNewsML.NewsContent.TaggedBody',
 'YNewsML.NewsContent.MultiMedia.DataFile.mimetype',
 'YNewsML.NewsContent.MultiMedia.DataFile.type',
 'YNewsML.NewsContent.MultiMedia.DataFile.content',
 'YNewsML.NewsContent.MultiMedia.DataFile.desc',
 'YNewsML.NewsContent.Comment',
 'YNewsML.Metadata.Region.RegionCode',
 'YNewsML.Metadata.Attribute.AttributeCode',
 'YNewsML.RefContent',
 'YNewsML.Metadata.Issue.IssueCode.code',
 'YNewsML.Metadata.Issue.IssueCode.name',
 'YNewsML.RefContent.Title',
 'YNewsML.RefContent.Href',
 'YNewsML.RefContent.ContentID',
 'YNewsML.RefContent.type',
 'YNewsML.Metadata.Embargo']